The following code illustrates:
1. Usage of ParDo (implementation)
2. Composite transforms and its implementation

In [ ]:
!pip install --quiet apache-beam[gcp]

In [ ]:
from google.colab import files
files.upload()

In [28]:
import apache_beam as beam

class Splitrows(beam.DoFn):
  def process(self,element):
    return [element.split(',')]

class FilterAccounts(beam.DoFn):
  def process(self,element):
    if element[3] == 'Accounts':
      return [element]

class FilterHR(beam.DoFn):
  def process(self,element):
    if element[3] == 'HR':
      return [element]

class PairElement(beam.DoFn):
  def process(self,element):
    return [(element[1]+','+element[3],1)]

class SumElements(beam.DoFn):
  def process(self,element):
    (key,value) = element
    return [(key,sum(value))]

class MyTransform(beam.PTransform):
  def expand(self,input_col):
    return (input_col|beam.ParDo(PairElement())
    |beam.GroupByKey()
    |beam.ParDo(SumElements()))

p1=beam.Pipeline()
input_collection=(p1
                  |beam.io.ReadFromText('/content/dept_data.txt')
                  |beam.ParDo(Splitrows())
                  # |beam.io.WriteToText('/content/output')
)
account_collection=(input_collection
                    |'filter accounts records'>>beam.ParDo(FilterAccounts())
                    # |'Pair to 1' >> beam.ParDo(PairElement())
                    # |'Group by key' >> beam.GroupByKey()
                    # |'sum values of each key'>>beam.ParDo(SumElements())
                    |'account transform' >> MyTransform()
                    |'write accounts' >> beam.io.WriteToText('/content/output-account')
)
hr_collection=(input_collection
                    |'filter hr records'>> beam.ParDo(FilterHR())
                    # |'Pair to one'>>beam.ParDo(PairElement())
                    # |'Group by HR key'>>beam.GroupByKey()
                    # |'sum values of each HR key'>>beam.ParDo(SumElements())
                    |'HR transform' >> MyTransform()
                    |'write HR'>>beam.io.WriteToText('/content/output-hr')
)
p1.run()

!{('head -n 20 /content/output*')}

==> /content/output-account-00000-of-00001 <==
('Marco,Accounts', 31)
('Rebekah,Accounts', 31)
('Itoe,Accounts', 31)
('Edouard,Accounts', 31)
('Kyle,Accounts', 62)
('Kumiko,Accounts', 31)
('Gaston,Accounts', 31)
('Ayumi,Accounts', 30)

==> /content/output-hr-00000-of-00001 <==
('Beryl,HR', 62)
('Olga,HR', 31)
('Leslie,HR', 31)
('Mindy,HR', 31)
('Vicky,HR', 31)
('Richard,HR', 31)
('Kirk,HR', 31)
('Kaori,HR', 31)
('Oscar,HR', 31)
